# This notebook serves for the purpose of testing code
# The content will be transfered to run.py for submission and evaluation

In [61]:
import numpy as np
import time
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.utils.rendertools import RenderTool
from flatland.envs.observations import GlobalObsForRailEnv
from flatland.envs.observations import TreeObsForRailEnv
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_env import GridTransitionMap


In [62]:
NUMBER_OF_AGENTS = 10

nr_start_goal = 10
nr_extra = 2
min_dist = 8
max_dist = 99999
# seed = 0
seed = np.random.randint(0,1000)

In [63]:
my_rail_generator = complex_rail_generator (
        nr_start_goal,
        nr_extra,
        min_dist,
        max_dist,
        seed
    )

env = RailEnv(
    
    # parameters taht control the dimensions of the map.
    width = 30,
    height = 30,
    
    # Rail generator: generates new rail networks on each reset
    # simplest rail generators: 
    #         envs.rail_generators.rail_from_manual_specifications_generator and 
    #         envs.rail_generators.random_rail_generator.
    
    rail_generator = my_rail_generator,
    schedule_generator = complex_schedule_generator(),
    obs_builder_object= GlobalObsForRailEnv(),
    number_of_agents = NUMBER_OF_AGENTS
)


In [64]:
# The original railway map from the complex_rail_generator
original_rail_map = env.rail.grid.copy()




In [65]:
# straightward implementation, could be optimized
# The node id is in the form (x,y), could be a linear mapping from 
# the most leftupper cornner to the most rightlower corner

# Flatland map encoding scheme, NESW -> N:[NESW], E:[NESW], S[NESW], W[NESW]
# Deadend cell is different, agent faces the opposite to the direction that it can goes. 


class decode_map:
    
    
    def __init__(self,original_map):
        self.original_map = original_map
        self.converted_input = {}

    # Determine next nodes for current_node.
    def case_matching(self, current_node, short_bits):
        
        next_nodes = []
        
        if (int(short_bits[0]) == 1):
            
            next_node = (current_node[0]-1, current_node[1])
#             print("N", next_node)
            
            next_nodes.append(next_node)
        
        if(int(short_bits[1]) == 1):
            
            next_node = (current_node[0], current_node[1]+1)
#             print("E", next_node)
            
            next_nodes.append(next_node)

        
        if(int(short_bits[2]) == 1):
            
            next_node = (current_node[0]+1, current_node[1])
#             print("S", next_node)
            
            next_nodes.append(next_node)

        if(int(short_bits[3]) == 1):
            
            next_node = (current_node[0], current_node[1]-1)
#             print("W", next_node)
            
            next_nodes.append(next_node)
        
        return next_nodes
        
    # First find the possible previous nodes, and call another function to find the possible next nodes.
    def slice_16_bits(self, current_node, bits):
        
        previous_nodes = {}
        
        # Facing North
        
        if (bits[0:4] != "0000"):
            
#             print(bits[0:4])
            next_nodes = self.case_matching(current_node, bits[0:4])
#             print("N, and returned next nodes", next_nodes)
            
            # add one previous node, and what nodes the agent can go from current node.
            previous_nodes[(current_node[0]+1,current_node[1])] = next_nodes
            
#             print("N, and current previous nodes", previous_nodes)
            
        # East
        if (bits[4:8] != "0000"):

#             print(bits[4:8])
            next_nodes = self.case_matching(current_node, bits[4:8])
#             print("E",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]-1)] = next_nodes
            
#             print("E, and current previous nodes", previous_nodes)
# 

        # South
        if (bits[8:12] != "0000"):
            
#             print(bits[8:12])
            next_nodes = self.case_matching(current_node, bits[8:12])
#             print("S",next_nodes)
            
            previous_nodes[(current_node[0]-1, current_node[1])] = next_nodes
            
#             print("S, and current previous nodes", previous_nodes)
       
        # West
        if (bits[12:16] != "0000"):
            
#             print(bits[12:16])
            next_nodes = self.case_matching(current_node, bits[12:16])
#             print("W",next_nodes)
            
            previous_nodes[(current_node[0], current_node[1]+1)] = next_nodes
            
#             print("W, and current previous nodes", previous_nodes)

        return previous_nodes
            
    def convert_ori_rail_map(self):
        for row in range(0,len(self.original_map)):
            for col in range(0,len(self.original_map[row])):
                
                # Current node: (row,col)
                print("Current node:", (row,col))
                current_node = (row,col)
                # Get previous node, decided by which direction the agent is facing.
                # EXCEPT the deadend nodes.
                
                
#                 print('{0:016b}'.format(self.original_map[row][col]))
                previous_next_nodes = self.slice_16_bits(current_node, '{0:016b}'.format(self.original_map[row][col]))
#                 print(previous_next_nodes.items())
                
                for previous,direction in previous_next_nodes.items():
#                     print("direction", direction)

                    self.converted_input[(current_node, previous)] = direction
#         print(self.converted_input)
        return self.converted_input

In [66]:
result = decode_map(original_rail_map).convert_ori_rail_map()

Current node: (0, 0)
Current node: (0, 1)
Current node: (0, 2)
Current node: (0, 3)
Current node: (0, 4)
Current node: (0, 5)
Current node: (0, 6)
Current node: (0, 7)
Current node: (0, 8)
Current node: (0, 9)
Current node: (0, 10)
Current node: (0, 11)
Current node: (0, 12)
Current node: (0, 13)
Current node: (0, 14)
Current node: (0, 15)
Current node: (0, 16)
Current node: (0, 17)
Current node: (0, 18)
Current node: (0, 19)
Current node: (0, 20)
Current node: (0, 21)
Current node: (0, 22)
Current node: (0, 23)
Current node: (0, 24)
Current node: (0, 25)
Current node: (0, 26)
Current node: (0, 27)
Current node: (0, 28)
Current node: (0, 29)
Current node: (1, 0)
Current node: (1, 1)
Current node: (1, 2)
Current node: (1, 3)
Current node: (1, 4)
Current node: (1, 5)
Current node: (1, 6)
Current node: (1, 7)
Current node: (1, 8)
Current node: (1, 9)
Current node: (1, 10)
Current node: (1, 11)
Current node: (1, 12)
Current node: (1, 13)
Current node: (1, 14)
Current node: (1, 15)
Current 

In [67]:
# pretty print result

for i in result.items():
    print(i,"\n")

(((1, 14), (2, 14)), [(2, 14)]) 

(((2, 14), (3, 14)), [(1, 14)]) 

(((2, 14), (1, 14)), [(3, 14)]) 

(((3, 14), (2, 14)), [(3, 15)]) 

(((3, 14), (3, 15)), [(2, 14)]) 

(((3, 15), (3, 14)), [(3, 16)]) 

(((3, 15), (3, 16)), [(3, 14)]) 

(((3, 16), (4, 16)), [(3, 15)]) 

(((3, 16), (3, 15)), [(4, 16)]) 

(((4, 1), (4, 2)), [(4, 2)]) 

(((4, 2), (5, 2)), [(4, 1)]) 

(((4, 2), (4, 1)), [(5, 2)]) 

(((4, 16), (5, 16)), [(3, 16)]) 

(((4, 16), (3, 16)), [(5, 16)]) 

(((5, 2), (6, 2)), [(4, 2)]) 

(((5, 2), (4, 2)), [(6, 2)]) 

(((5, 13), (5, 14)), [(5, 14)]) 

(((5, 14), (5, 13)), [(5, 15)]) 

(((5, 14), (5, 15)), [(5, 13)]) 

(((5, 15), (5, 14)), [(5, 16)]) 

(((5, 15), (5, 16)), [(5, 14)]) 

(((5, 16), (6, 16)), [(4, 16)]) 

(((5, 16), (5, 15)), [(5, 17)]) 

(((5, 16), (4, 16)), [(6, 16)]) 

(((5, 16), (5, 17)), [(5, 15)]) 

(((5, 17), (5, 16)), [(5, 18)]) 

(((5, 17), (5, 18)), [(5, 16)]) 

(((5, 18), (6, 18)), [(5, 17)]) 

(((5, 18), (5, 17)), [(6, 18)]) 

(((6, 2), (5, 2)), [(6, 3)]) 

In [68]:
# Obtain the start and target locations
# Assuming that each agent is going from one assigned start location to one fixed target location.
# i.e. no multiple targets available for one agent to go to.

# can be access by env.agents (lists of agents objects with their own information)
'''
informaiton include: poistion (i.e. start location at time step 0), 
                     direction,
                     target,
                     speed_data,
                     malfunction_data,
                     handle,
                     old_direction,
                     old_position     
'''

agent_info = {}

for index, agent in enumerate(env.agents):
    print(index, agent)
   

0 EnvAgent(position=(12, 2), direction=0, target=(19, 28), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=0, old_direction=None, old_position=None)
1 EnvAgent(position=(20, 0), direction=3, target=(25, 22), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=1, old_direction=None, old_position=None)
2 EnvAgent(position=(22, 29), direction=1, target=(17, 11), moving=False, speed_data={'position_fraction': 0.0, 'speed': 1.0, 'transition_action_on_cellexit': 0}, malfunction_data={'malfunction': 0, 'malfunction_rate': 0.0, 'next_malfunction': 0, 'nr_malfunctions': 0}, handle=2, old_direction=None, old_position=None)
3 EnvAgent(position=(25, 11), direction=2, ta

In [69]:
# idx2node: {index:(current, previous)}
idx2node = {idx:k for idx, k in enumerate(result.keys())} 

# node2idx: {(current, previous) : index}
node2idx = {k:idx for idx, k in idx2node.items()}

# idx2pos: {index:current}
idx2pos = {k:v[0] for (k,v) in idx2node.items()}

edges = []
for cur, prev in result:
    print (cur, prev, result[(cur, prev)])
    for to in result[(cur, prev)]:
        print(to)
        edges.append((node2idx[(cur, prev)], node2idx[(to, cur)]))
        
        
def linearize_loc(env, loc):
    return loc[0]*env.width + loc[1]

(1, 14) (2, 14) [(2, 14)]
(2, 14)
(2, 14) (3, 14) [(1, 14)]
(1, 14)
(2, 14) (1, 14) [(3, 14)]
(3, 14)
(3, 14) (2, 14) [(3, 15)]
(3, 15)
(3, 14) (3, 15) [(2, 14)]
(2, 14)
(3, 15) (3, 14) [(3, 16)]
(3, 16)
(3, 15) (3, 16) [(3, 14)]
(3, 14)
(3, 16) (4, 16) [(3, 15)]
(3, 15)
(3, 16) (3, 15) [(4, 16)]
(4, 16)
(4, 1) (4, 2) [(4, 2)]
(4, 2)
(4, 2) (5, 2) [(4, 1)]
(4, 1)
(4, 2) (4, 1) [(5, 2)]
(5, 2)
(4, 16) (5, 16) [(3, 16)]
(3, 16)
(4, 16) (3, 16) [(5, 16)]
(5, 16)
(5, 2) (6, 2) [(4, 2)]
(4, 2)
(5, 2) (4, 2) [(6, 2)]
(6, 2)
(5, 13) (5, 14) [(5, 14)]
(5, 14)
(5, 14) (5, 13) [(5, 15)]
(5, 15)
(5, 14) (5, 15) [(5, 13)]
(5, 13)
(5, 15) (5, 14) [(5, 16)]
(5, 16)
(5, 15) (5, 16) [(5, 14)]
(5, 14)
(5, 16) (6, 16) [(4, 16)]
(4, 16)
(5, 16) (5, 15) [(5, 17)]
(5, 17)
(5, 16) (4, 16) [(6, 16)]
(6, 16)
(5, 16) (5, 17) [(5, 15)]
(5, 15)
(5, 17) (5, 16) [(5, 18)]
(5, 18)
(5, 17) (5, 18) [(5, 16)]
(5, 16)
(5, 18) (6, 18) [(5, 17)]
(5, 17)
(5, 18) (5, 17) [(6, 18)]
(6, 18)
(6, 2) (5, 2) [(6, 3)]
(6, 3)
(6, 

In [70]:
# convert agent start information from the format "start_position, direction = a number"
# to edge

start_idx = []
for agent in env.agents:
    print(agent.position)
    print(agent.direction) # direction encoding: {0: North, 1: East, 2: South, 3: West}
    current_pos = agent.position
    if(agent.direction ==0):
        prev_pos = (current_pos[0]+1,current_pos[1])
    elif(agent.direction == 1):
        prev_pos = (current_pos[0],current_pos[1]-1)
    elif(agent.direction == 2):
        prev_pos = (current_pos[0]-1,current_pos[1])
    elif(agent.direction == 3):
        prev_pos = (current_pos[0], current_pos[1]+1)
    start_node = (current_pos,prev_pos)
    index = node2idx[start_node]
    
    start_idx.append(index)
#     print(start_node, index)

print(start_idx)

(12, 2)
0
(20, 0)
3
(22, 29)
1
(25, 11)
2
(11, 28)
1
(6, 15)
0
(26, 2)
3
(6, 21)
1
(17, 5)
2
(22, 22)
2
[86, 293, 391, 417, 85, 37, 419, 46, 218, 378]


In [71]:
pos2nodes = dict()
for _, (cur, prev) in idx2node.items():
    if cur not in pos2nodes:
        pos2nodes[cur] = []
    pos2nodes[cur].append((cur, prev))

In [72]:
target_idx = []
for i,agent in enumerate(env.agents):
    goal_nodes = pos2nodes[agent.target]
    # print(goal_nodes)
    if len(goal_nodes) > 1:
        print("Goal node is not a deadend")
    
    target_idx.append(node2idx[goal_nodes[0]])

In [73]:
# map file

node_file = ""
edge_file = ""


node_file += str(len(idx2node)) + "\n"
for idx,node in idx2node.items():
    node_file += str(linearize_loc(env, idx2pos[idx])) + "\n"

edge_file += str(len(edges)) + "\n"
for edge in edges:
    edge_file += str(edge[0]) + "," + str(edge[1]) +"\n"

map_file = node_file + edge_file

file = open("map.txt", "w") 
file.write(map_file) 
file.close() 

In [74]:
# agent file
start_goal_pos_file = ""
start_goal_pos_file += str(len(env.agents)) + "\n"
for i,agent in enumerate(env.agents):
    start_goal_pos_file +=  str(start_idx[i]) + "," + str(target_idx[i]) +",1\n"
    

file = open("agents.txt", "w") 
file.write(start_goal_pos_file) 
file.close()

In [53]:
# create local files


node_file = ""
current_node_file = ""
edge_file = ""
start_goal_pos_file = ""

node_file += str(len(idx2node)) + "\n"
for idx,node in idx2node.items():
    node_file += str(idx)+ "," + str(node) + "\n"

current_node_file += str(len(idx2node)) + "\n"
for idx,node in idx2node.items():
    current_node_file += str(idx)+ "," + str(node[0]) + "\n"
    
edge_file += str(len(edges)) + "\n"
for edge in edges:
    edge_file += str(edge) +"\n"

start_goal_pos_file += str(len(env.agents)) + "\n"
for i,agent in enumerate(env.agents):
    start_goal_pos_file += str(i) + "," + str(start_idx[i]) + ", " + str(agent.target) +"\n"
    
    
file = open("idx_node.txt", "w") 
file.write(node_file) 
file.close() 

file = open("idx_curr_node.txt", "w") 
file.write(current_node_file) 
file.close()

file = open("edges.txt", "w") 
file.write(edge_file) 
file.close()


file = open("start_goals.txt", "w") 
file.write(start_goal_pos_file) 
file.close()

In [ ]:
# tool to render environments
env_renderer = RenderTool(env, gl="PIL")
# env_renderer = RenderTool(env)

# env_renderer = RenderTool (env)
env_renderer.render_env(show=True)



In [ ]:
import os
nowtime = os.popen('g++ ./main.cpp')
result = os.popen('./a.out')
print (result.read())

In [ ]:
import threading

def work1():
    os.popen('g++ ./main.cpp -o a.out')
    result = os.popen('./a.out')
    print(result.read())
def work2():
    os.popen('g++ ./main.cpp -o a1.out')
    result = os.popen('./a1.out')
    print(result.read())
    return result 
    
t1 = threading.Thread(target=work1)
t2 = threading.Thread(target=work2)

t1.start()
t2.start()

In [ ]:
import threading
 
def work1: 
    for i in range(x,y):
        print(i)
 
ta = threading.Thread(target=threadfun,args=(1,6))      
tb = threading.Thread(target=threadfun,args=(10,15))    
ta.start()          


In [9]:
def my_controller():
    """
    You are supposed to write this controller
    """
    _action = {}
    for _idx in range(NUMBER_OF_AGENTS):
        _action[_idx] = np.random.randint(0, 5)
    return _action

for step in range(1):

    # Get agents' handles to give actions
    # handles = env.get_agent_handles()
    
    # e.g. giving two agents actions. 
    # action_dict = {handles[0]:0, handles[1]:0}
    
    _action = my_controller()
    
    # obs, all_rewards, done: dictionary indexed by agents handles
        # values: correspond to the relevant obeservations, rewards and terminal 
        #       status for each agent.
        
    obs, all_rewards, done, _ = env.step(_action) # environment take one step with the provided actions.
    
    # show results
    print("Observation: \n", obs)
    print("Rewards: {}, [done={}]".format( all_rewards, done))
    env_renderer.render_env(show=True, frames=False, show_observations=False)
    time.sleep(1)

Observation: 
 {0: (array([[[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1.,

NameError: name 'env_renderer' is not defined

In [75]:
import os
import subprocess

result = os.popen("./ECBS/ECBS -m ./map.txt  -a ./agents.txt -o ./paths.txt -t 60 -w 1.2 --makespan 1000")
# result = os.popen()
print (result.read())

ECBS: 
        Succeed ; cost: 220, ;lowerbound: 219; preTime: 0.003253; runtime: 8e-06; nodes:   0



In [76]:
with open("paths.txt", "r") as f:
    docs = f.readlines()
    
def parse_line_path(l):
    return [int(node) for node in l.split(",")[:-1]]

paths = [parse_line_path(l) for l in docs]

In [77]:
start_idx

[86, 293, 391, 417, 85, 37, 419, 46, 218, 378]

In [78]:
target_idx

[292, 418, 223, 9, 279, 410, 87, 16, 94, 0]

In [87]:
# check connectivity
for path in paths:
    locations = [idx2node[p] for p in path]
    print(locations)
    for i in range(len(locations) - 1):
        if locations[i][0] != locations[i + 1][1]:
            print("error")

[((12, 2), (13, 2)), ((13, 2), (12, 2)), ((13, 3), (13, 2)), ((13, 4), (13, 3)), ((14, 4), (13, 4)), ((14, 5), (14, 4)), ((14, 6), (14, 5)), ((14, 7), (14, 6)), ((14, 8), (14, 7)), ((14, 9), (14, 8)), ((14, 10), (14, 9)), ((15, 10), (14, 10)), ((15, 11), (15, 10)), ((15, 12), (15, 11)), ((15, 13), (15, 12)), ((15, 14), (15, 13)), ((16, 14), (15, 14)), ((16, 15), (16, 14)), ((16, 16), (16, 15)), ((16, 17), (16, 16)), ((16, 18), (16, 17)), ((16, 19), (16, 18)), ((17, 19), (16, 19)), ((17, 20), (17, 19)), ((17, 21), (17, 20)), ((17, 22), (17, 21)), ((17, 23), (17, 22)), ((17, 24), (17, 23)), ((17, 25), (17, 24)), ((18, 25), (17, 25)), ((18, 26), (18, 25)), ((18, 27), (18, 26)), ((18, 28), (18, 27)), ((19, 28), (18, 28))]
[((20, 0), (20, 1)), ((20, 1), (20, 0)), ((20, 2), (20, 1)), ((20, 3), (20, 2)), ((20, 4), (20, 3)), ((20, 5), (20, 4)), ((21, 5), (20, 5)), ((21, 6), (21, 5)), ((21, 7), (21, 6)), ((21, 8), (21, 7)), ((21, 9), (21, 8)), ((22, 9), (21, 9)), ((22, 10), (22, 9)), ((22, 11),

In [85]:
location_paths = [[idx2pos[p] for p in path] for path in paths]

In [86]:
for i in range(len(location_paths) - 1):
    for j in range(i + 1,len(location_paths)):
        for p1, p2 in zip(location_paths[i], location_paths[j]):
            if (p1 == p2):
                print("error")

In [83]:
lo

NameError: name 'lo' is not defined

In [84]:
paths

[[86,
  105,
  107,
  110,
  138,
  140,
  143,
  147,
  150,
  152,
  155,
  176,
  178,
  180,
  182,
  185,
  201,
  203,
  206,
  208,
  210,
  214,
  238,
  240,
  242,
  244,
  246,
  248,
  251,
  267,
  269,
  271,
  274,
  292],
 [293,
  294,
  296,
  298,
  301,
  305,
  328,
  330,
  332,
  334,
  338,
  352,
  354,
  356,
  358,
  360,
  363,
  366,
  368,
  370,
  372,
  374,
  377,
  402,
  405,
  411,
  414,
  418],
 [391,
  390,
  388,
  386,
  384,
  382,
  380,
  347,
  324,
  323,
  321,
  319,
  316,
  313,
  311,
  288,
  263,
  262,
  260,
  232,
  231,
  227,
  225,
  223],
 [417,
  408,
  398,
  397,
  395,
  351,
  337,
  336,
  306,
  277,
  254,
  221,
  220,
  198,
  174,
  146,
  114,
  90,
  77,
  71,
  70,
  63,
  57,
  56,
  47,
  35,
  34,
  32,
  30,
  14,
  10,
  9],
 [85,
  84,
  104,
  102,
  100,
  136,
  134,
  132,
  130,
  128,
  125,
  123,
  164,
  191,
  188,
  211,
  209,
  207,
  236,
  236,
  234,
  259,
  287,
  285,
  281,
  279],
 [37,
